In [17]:
import pandas as pd
import numpy as np
from blimpy import Waterfall
import cv2 as cv

In [11]:
def normalize(data):
    epsilon = 1
    min_val = data.min()
    data = data - min_val + epsilon
    new_data = np.log(data)
    min_val = data.min()
    max_val = data.max()
    final_data = (data - min_val) / (max_val - min_val)
    return final_data

In [24]:
import pickle

with open('SOI.pkl', 'rb') as handle:
    SOI = pickle.load(handle)

target1 = SOI["GBT_57907_34727_HIP87579_fine"]
print(target1)
data = Waterfall("../../../../../../../"+target1["directory"], 
                 f_start = target1["start"], f_stop=target1["end"]).data
data = normalize(data)*255
data = data.astype('uint8')
data = np.swapaxes(data, 1, -1)
sift = cv.SIFT_create()
kp, des = sift.detectAndCompute(data,None)
print(des.shape)

{'freq': 1961.273217201233, 'start': 1961.2728595733643, 'end': 1961.2735748291016, 'directory': '/mnt_blpd7/datax/dl/GBT_57907_34727_HIP87579_fine.h5', 'stats': 3280.7677441581645}
(1, 128)


In [25]:
total_data = Waterfall("../../../../../../../"+target1["directory"], max_load=False).data

In [ ]:
from tqdm import tqdm
num_samples = total_data.shape[-1]//256
new_total_data = np.zeros((num_samples, 16, 256))

des_vecs = []
kps_vecs = []
sift = cv.SIFT_create()

for i in tqdm(range(num_samples)):
    temp =  normalize(total_data[:,0,i*256:(i+1)*256]) *255
    new_total_data[i, :,:] =temp
    temp = temp.astype('uint8')
    kp, des = sift.detectAndCompute(temp, None)
    des_vecs.append(des)
    kps_vecs.append(kp)

 31%|█████████████████████▎                                              | 441368/1404928 [11:14<23:33, 681.50it/s]

In [ ]:
descriptors = np.float32(des_vecs)

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 5, .01)
centroids = cv.kmeans(descriptors, clusters, None, criteria, 1, cv.KMEANS_PP_CENTERS)[2]